# Using Hugging Face With Plugins

In this notebook, we demonstrate using Hugging Face models for Plugins using both SemanticMemory and text completions. 

SK supports downloading models from the Hugging Face that can perform the following tasks: text-generation, text2text-generation, summarization, and sentence-similarity. You can search for models by task at https://huggingface.co/models.

In [ ]:
!python -m pip install semantic-kernel==0.5.1.dev0

# Note that additional dependencies are required for the Hugging Face connectors:
!python -m pip install torch==2.0.0
!python -m pip install transformers==^4.28.1
!python -m pip install sentence-transformers==^2.2.2

In [1]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.hugging_face as sk_hf

In [2]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.HuggingFace

First, we will create a kernel and add both text completion and embedding services. 

For text completion, we are choosing GPT2. This is a text-generation model. (Note: text-generation will repeat the input in the output, text2text-generation will not.)
For embeddings, we are using sentence-transformers/all-MiniLM-L6-v2. Vectors generated for this model are of length 384 (compared to a length of 1536 from OpenAI ADA).

The following step may take a few minutes when run for the first time as the models will be downloaded to your local machine.

In [3]:
kernel = sk.Kernel()

# Configure LLM service
if selectedService == Service.HuggingFace:
    text_service_id = "gpt2"
    kernel.add_service(
        service=sk_hf.HuggingFaceTextCompletion(
            service_id=text_service_id, ai_model_id=text_service_id, task="text-generation"
        ),
    )
    embed_service_id = "sentence-transformers/all-MiniLM-L6-v2"
    embedding_svc = sk_hf.HuggingFaceTextEmbedding(service_id=embed_service_id, ai_model_id=embed_service_id)
    kernel.add_service(
        service=embedding_svc,
    )
    kernel.use_memory(storage=sk.memory.VolatileMemoryStore(), embeddings_generator=embedding_svc)
    kernel.import_plugin(sk.core_plugins.TextMemoryPlugin(), "TextMemoryPlugin")

### Add Memories and Define a plugin to use them

Most models available on huggingface.co are not as powerful as OpenAI GPT-3+. Your plugins will likely need to be simpler to accommodate this.

In [4]:
await kernel.memory.save_information(collection="animal-facts", id="info1", text="Sharks are fish.")
await kernel.memory.save_information(collection="animal-facts", id="info2", text="Whales are mammals.")
await kernel.memory.save_information(collection="animal-facts", id="info3", text="Penguins are birds.")
await kernel.memory.save_information(collection="animal-facts", id="info4", text="Dolphins are mammals.")
await kernel.memory.save_information(collection="animal-facts", id="info5", text="Flies are insects.")

# Define prompt function using SK prompt template language
my_prompt = """I know these animal facts: {{recall $query1}} {{recall $query2}} {{recall $query3}} and """

execution_settings = sk_hf.HuggingFacePromptExecutionSettings(
    service_id=text_service_id,
    ai_model_id=text_service_id,
    max_tokens=45,
    temperature=0.5,
    top_p=0.5,
)

prompt_template_config = sk.PromptTemplateConfig(
    template=my_prompt,
    name="text_complete",
    template_format="semantic-kernel",
    execution_settings=execution_settings,
)

my_function = kernel.create_function_from_prompt(prompt_template_config=prompt_template_config)

Let's now see what the completion looks like! Remember, "gpt2" is nowhere near as large as ChatGPT, so expect a much simpler answer.

In [5]:
from semantic_kernel.functions import KernelArguments

arguments = KernelArguments(
    query1="animal that swims",
    query2="animal that flies",
    query3="penguins are?",
    collection="animal-facts",
    relevance=0.3,
)

output = await kernel.invoke(
    my_function,
    arguments,
)

output = str(output).strip()

query_result1 = await kernel.memory.search(
    collection="animal-facts", query=arguments["query1"], limit=1, min_relevance_score=0.3
)
query_result2 = await kernel.memory.search(
    collection="animal-facts", query=arguments["query2"], limit=1, min_relevance_score=0.3
)
query_result3 = await kernel.memory.search(
    collection="animal-facts", query=arguments["query3"], limit=1, min_relevance_score=0.3
)

print(f"{text_service_id} completed prompt with: '{output}'")

/home/evmattso/.cache/pypoetry/virtualenvs/semantic-kernel-eoLGgW5m-py3.10/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/evmattso/.cache/pypoetry/virtualenvs/semantic-kernel-eoLGgW5m-py3.10/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect par

gpt2 completed prompt with: ''
